In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forest_risks import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forest_risks.utils import get_store
import altair as alt
alt.themes.enable("carbonplan_light")

ThemeRegistry.enable('carbonplan_light')

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
coarsen = 4
mask = (
    (
        load.nlcd(store="az", year=2001).sel(band=[41, 42, 43, 90]).sum("band")
        > 0.25
    )
    .astype("float")
    .coarsen(x=coarsen, y=coarsen, boundary="trim")
    .mean()
)

In [4]:
historical_fire = xr.open_zarr(
    get_store("carbonplan-scratch", "data/fire_historical_v3.zarr")
)
fire_mask = ~np.isnan(historical_fire.historical.isel(time=0).drop("time"))

In [5]:
ds = (
    xr.open_zarr(get_store("carbonplan-scratch", "data/fire_future_v3.zarr"))
    .assign_coords({"x": mask.x, "y": mask.y})
    .where(fire_mask)
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [6]:
ds_remaining = (
    xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_future_v3_remaining.zarr")
    )
    .assign_coords({"x": mask.x, "y": mask.y})
    .groupby("time.year")
    .sum()
    .where(fire_mask)
    .compute()
)

In [7]:
ds = xr.merge([ds, ds_remaining])

In [8]:
future_maps = ds.sel(year=slice("2060", "2089")).mean(dim="year").compute()
future_ts = ds.mean(dim=["x", "y"]).compute()

In [9]:
gcms = [
    ("MRI-ESM2-0", (0, 0)),
    ("MIROC-ES2L", (1, 0)),
    ("MPI-ESM1-2-LR", (2, 0)),
    ("ACCESS-ESM1-5", (3, 0)),
    ("ACCESS-CM2", (4, 0)),
    ("CanESM5-CanOE", (5, 0)),
]
titles = [
    "Burn area\n[fraction/year]",
    "Drought mortality\n[]",
    "Biotic agent mortality\n[]",
]

In [10]:
scenario_dict = {}
future_maps = ds.sel(year=slice("2060", "2089")).mean(dim="year").compute()
for scenario in ["ssp245", "ssp370", "ssp585"]:
    scenario_dict[scenario] = (
        future_maps[[gcm + "_" + scenario for gcm, _x in gcms]]
        .to_array(dim="vars")
        .mean(dim="vars")
    )

In [13]:
figure = alt.hconcat()
for scenario in ["ssp245", "ssp370", "ssp585"]:
    fire = plot.fire.simple_map(
        scenario_dict[scenario],
        clim=(0.001, 0.03),
        clabel="Burn area (fraction/year)",
        cmap=['#FFC59E', '#C4550A'],
        title1=scenario,
    )
    figure |= fire
figure

alt.HConcatChart(...)